In [2]:
import pandas as pd
from IPython.core.debugger import set_trace
import json
import re
import os
import time
import requests
import csv
from twython import Twython, TwythonRateLimitError, TwythonError
from app.models import Brand, Profile, Hashtag
from custom_user.models import EmailUser
from tqdm.autonotebook import tqdm
tqdm.pandas()

# Scraping

In [79]:
df = pd.read_excel('../texcrapy/keywords and logos.xlsx', 
                   sheet_name='20190304')[['shortname','kw_supporter','kw_supported','keywords']][3:];df
_or = lambda kw: ' OR '.join(['#' + k.strip() for k in kw.split(',')])

qry_base = {row.shortname:_or(row.keywords) for row in df.itertuples()}
supporters = df.shortname[df.kw_supporter==True]
qry_sup = ' OR '.join([qry_base[sup] for sup in supporters]); qry_sup
qry = {row.shortname: '(' + qry_base[row.shortname] + ') AND (' + qry_sup + ')' if row.kw_supported==True else qry_base[row.shortname] for row in df.itertuples()}

In [58]:
%%time
what = ['id']#, 'likes', 'text', 'timestamp', 'user']
texcrapy.scrap(qry, what=what, lang='ko', start='2019-01-01', end='2019-06-20', download_to='scrapped/getch')

NameError: name 'texcrapy' is not defined

# 스크랩 파일명 가져오기

In [3]:
from glob import glob
glob_pattern = os.path.join('../texcrapy/scrapped/twitter_getch', '*')
fnames = glob(glob_pattern)
# fnames = ['scrapped/twitter_getch/' + fname for fname in os.listdir('../texcrapy/scrapped/twitter_getch')]

In [4]:
tweet_ids = {}
for fname in tqdm(fnames[:]):
    with open(fname, encoding='UTF-8-sig') as f:
        for item, docs in json.load(f).items():
            if len(docs) > 0:
                tweet_ids[item] = set()
                for doc in docs: tweet_ids[item].add(doc['id'])

In [6]:
tweet_ids;

# 트위터 크롤링 (by ID)

In [6]:
def get_hashtags(tweet):
    _hashtags = tweet['entities']['hashtags']
    
    if len(_hashtags) > 0:
        return {ht['text'] for ht in _hashtags}
    else:
        return set()
                

def get_user(tweet):
    _user = tweet['user']
    
    user = {}
    user['user'] = _user['screen_name']

    if 'profile_image_url' in _user:
        user['profile_image'] = _user['profile_image_url']
            
    return {_user['id']:user}
        
    
def get_feed_image(tweet):
    entities = tweet['entities']
    
    if 'media' in entities:
        media = entities['media']
        
        if len(media) > 0:
            _media = media[0]
            
            if 'media_url' in _media:
                return _media['media_url']
            
    return None
    

def get_feed(tweet, bname, hashtags):
    feed = {
        'membership': bname, 
        'author': tweet['user']['screen_name'], 
        'created_at': tweet['created_at'], 
        'nlikes': tweet['favorite_count'], 
        'content': tweet['text'],
        'hashtags': list(hashtags),
    }
    
    feed_image = get_feed_image(tweet)
    
    if feed_image is not None:
        feed['feed_image'] = feed_image
        
    return feed

In [16]:
CONSUMER_KEY = 'gFeP16lpEBKux4K6GO2LfLEWI'
CONSUMER_SECRET = '38yzPyI8vqUqrCV0RLc1ugl8Se1pYA2Nlderk9Rj3DUaE5VTpA'
OAUTH_TOKEN = '1085907708413267970-nyk4nvgG9miNTqRmg4jkFnmZfqW254'
OAUTH_TOKEN_SECRET = 'ty2TIFFSorxmMKMq0VlqcIWrgqpJqdHBW9z3ye0J4BeL7'
twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

feeds = []
users = dict()
hashtags = set()

for bname, ids in tqdm(list(tweet_ids.items())[:]):
    for tw_id in tqdm(ids, leave=False):
        while True:
            try:
                tw = twitter.show_status(id=tw_id)
                _hashtags = get_hashtags(tw)
                _user = get_user(tw)
                _feed = get_feed(tw, bname, _hashtags)

                hashtags |= _hashtags
                users.update(_user)
                feeds.append(_feed)

            except TwythonRateLimitError as e:
                print(bname, tw_id, e)
                remainder = float(twitter.get_lastfunction_header(header='x-rate-limit-reset')) - time.time()
                remainder = max(0, remainder)
                del twitter
                time.sleep(remainder)
                twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
                continue

            except TwythonError as e:
                print(bname, tw_id, e)

            break

8seconds 1138403305065893888 Twitter API returned a 404 (Not Found), Sorry, that page does not exist.


adidas 1112177329516748800 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
adidas 1120901613751660544 Twitter API returned a 403 (Forbidden), User has been suspended.
adidas 1132602291133882368 Twitter API returned a 404 (Not Found), No status found with that ID.
adidas 1101442105350606848 Twitter API returned a 404 (Not Found), No status found with that ID.
adidas 1129337167245824000 Twitter API returned a 403 (Forbidden), User has been suspended.
adidas 1098856237049864192 Twitter API returned a 404 (Not Found), No status found with that ID.
adidas 1109288860809416705 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
adidas 1097802347810443264 Twitter API returned a 404 (Not Found), No status found with that ID.
adidas 1110026329968336896 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
adidas 1098854458262339585 Twitter API returned a 404 (Not Found), No status 

adidasoriginal 1084131496913453056 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
adidasoriginal 1083962574394347522 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
adidasoriginal 1105483424230445058 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
adidasoriginal 1084325238572236800 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
adidasoriginal 1084449014928625667 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
adidasoriginal 1084420689753501696 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
adidasoriginal 1084319999035092992 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
adidasoriginal 1083986050073223170 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.


alexandermcqueen 1137389845762805760 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.


ami 1130714594924683265 Twitter API returned a 404 (Not Found), No status found with that ID.
ami 1138333544596398081 Twitter API returned a 404 (Not Found), No status found with that ID.
ami 1125398780004536325 Twitter API returned a 404 (Not Found), No status found with that ID.
ami 1130504197739204608 Twitter API returned a 404 (Not Found), No status found with that ID.
ami 1084025463490568193 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
ami 1102171799599513600 Twitter API returned a 404 (Not Found), No status found with that ID.
ami 1132608977416495104 Twitter API returned a 404 (Not Found), Sorry, that page does not exist.
ami 1131537900376547328 Twitter API returned a 404 (Not Found), Sorry, that page does not exist.
ami 1141348174763663360 Twitter API returned a 404 (Not Found), Sorry, that page does not exist.
ami 1141334290526945287 Twitter API returned a 404 (Not Found), No status found with that ID.
ami 1139523505492414464 Twitter

...

uniqlo 1098730101036933120 Twitter API returned a 404 (Not Found), No status found with that ID.
uniqlo 1106125713810358272 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
uniqlo 1098223831414534146 Twitter API returned a 404 (Not Found), No status found with that ID.
uniqlo 1132902119172739072 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
uniqlo 1130785624774963202 Twitter API returned a 404 (Not Found), No status found with that ID.
uniqlo 1106124461751914497 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
uniqlo 1106124460162199558 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
uniqlo 1106123638619729920 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
uniqlo 1106124977252823040 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
uniqlo 

valentino 1104195060067790849 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
valentino 1112618882034855937 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
valentino 1090516934855139328 Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
valentino 1090516934855139328 Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
valentino 1090516934855139328 Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
valentino 1090516934855139328 Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
valentino 1106062778748432384 Twitter API returned a 403 (Forbidden), User has been suspended.
valentino 1106063251345829889 Twitter API returned a 403 (Forbidden), User has been suspended.
valentino 1106066184615518208 Twitter API returned a 403 (Forbidden), User has been suspended.


vans 1115964097672048640 Twitter API returned a 404 (Not Found), No status found with that ID.
vans 1084457760979329024 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
vans 1120208499718287360 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
vans 1098166960401969152 Twitter API returned a 404 (Not Found), No status found with that ID.
vans 1098166391473991681 Twitter API returned a 404 (Not Found), No status found with that ID.
vans 1120103685671964673 Twitter API returned a 404 (Not Found), No status found with that ID.
vans 1115243600365797376 Twitter API returned a 404 (Not Found), No status found with that ID.


yeezy 1129285844999725056 Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.


zara 1098727636702965760 Twitter API returned a 404 (Not Found), No status found with that ID.
zara 1098730101036933120 Twitter API returned a 404 (Not Found), No status found with that ID.
zara 1098169828173537280 Twitter API returned a 404 (Not Found), No status found with that ID.
zara 1098369610158530561 Twitter API returned a 404 (Not Found), No status found with that ID.
zara 1097805506863804416 Twitter API returned a 404 (Not Found), No status found with that ID.


In [19]:
users;
with open('users.json', 'w') as f:
    json.dump(users, f)

In [27]:
hashtags;
with open('hashtags.json', 'w') as f:
    json.dump(list(hashtags), f)

In [31]:
feeds;
with open('feeds.json', 'w') as f:
    json.dump(feeds, f)

# Profile image 저장하기

In [24]:
users = None
with open('users.json', encoding='UTF-8-sig') as f:
    users = json.load(f)

In [22]:
for k,v in tqdm(users.items()):
    profile_dir = 'uploads/profile_images/' + v['user'] + '@getch.com'
    # profile_dir = 'uploads/profile_images/' + str(k)
    
    if not os.path.isdir(profile_dir):
        os.mkdir(profile_dir)
    
    profile_image = v['profile_image']
    fname = profile_dir + '/' + profile_image.split('/')[-1]

    if not os.path.isfile(fname):
        try:
            img = requests.get(profile_image).content
            with open(fname, 'wb') as f:
                f.write(img)
        except:
            print(v['user'])

# Feed image 저장하기

In [26]:
feeds = None
with open('feeds.json', encoding='UTF-8-sig') as f:
    feeds = json.load(f)

In [29]:
feeds;

In [28]:
for feed in tqdm(feeds):
    if 'feed_image' not in feed:
        continue
    
    feed_image = feed['feed_image']
    feed_memb_dir = 'uploads/feed_images/' + feed['membership']

    if not os.path.isdir(feed_memb_dir):
        os.mkdir(feed_memb_dir)
    
    feed_memb_author_dir = feed_memb_dir + '/' + feed['author'] + '@getch.com'
    
    if not os.path.isdir(feed_memb_author_dir):
        os.mkdir(feed_memb_author_dir)
    
    fname = feed_memb_author_dir + '/' + feed_image.split('/')[-1]

    if not os.path.isfile(fname):
        try:
            img = requests.get(feed_image).content
            with open(fname, 'wb') as f:
                f.write(img)
        except:
            print(fname)

uploads/feed_images/ami/bts_loverone@getch.com/D7Acz_6UcAEwPsd.jpg
uploads/feed_images/samsonite/eunhasoocake@getch.com/DxEoQwMU0AAB07p.jpg



# 테스트유저 만들기
## 1. Custom user model (email 기반)

In [3]:
from app.models import CustomEmailUser

In [4]:
users = None
with open('users.json', encoding='UTF-8-sig') as f:
    users = json.load(f)

In [5]:
users;

In [6]:
bulk_emailuser = []
for _id, _user in users.items():
    obj = CustomEmailUser()
    obj.id = _id
    obj.email = _user['user'] + '@getch.com'
    bulk_emailuser.append(obj)

CustomEmailUser.objects.bulk_create(bulk_emailuser)

[<CustomEmailUser: DazedKorea@getch.com>,
 <CustomEmailUser: 8056LpjXNJdgwuj@getch.com>,
 <CustomEmailUser: musinsacom@getch.com>,
 <CustomEmailUser: rvd3065@getch.com>,
 <CustomEmailUser: woodz_golmu@getch.com>,
 <CustomEmailUser: fashionncom@getch.com>,
 <CustomEmailUser: Lilith36910318@getch.com>,
 <CustomEmailUser: SeoulStore_twt@getch.com>,
 <CustomEmailUser: 4buynsell@getch.com>,
 <CustomEmailUser: kanghui__@getch.com>,
 <CustomEmailUser: agonyman@getch.com>,
 <CustomEmailUser: i_ilyo@getch.com>,
 <CustomEmailUser: jung_sang_won@getch.com>,
 <CustomEmailUser: thin_krungy0314@getch.com>,
 <CustomEmailUser: _feelthat2NE1@getch.com>,
 <CustomEmailUser: Denny19871207@getch.com>,
 <CustomEmailUser: yogipower@getch.com>,
 <CustomEmailUser: DongWoonK@getch.com>,
 <CustomEmailUser: pat5007@getch.com>,
 <CustomEmailUser: ZENITH_NEWS@getch.com>,
 <CustomEmailUser: tommygnosys@getch.com>,
 <CustomEmailUser: FuxxingFabulous@getch.com>,
 <CustomEmailUser: stussygo@getch.com>,
 <CustomEmailUse

## 2. Profile model

In [43]:
from app.models import Profile, CustomEmailUser

In [44]:
users = None
with open('users.json', encoding='UTF-8-sig') as f:
    users = json.load(f)

In [12]:
bulk_profile = []
for i, (_id, _user) in enumerate(tqdm(list(users.items())[:])):
    obj = Profile()
    obj.id = _id
    # obj.user = CustomEmailUser.objects.get(pk=_id)
    obj.user = bulk_emailuser[i]
    obj.profile_image = 'profile_images/' + _user['user'] + '@getch.com' + '/' + _user['profile_image'].split('/')[-1]
    bulk_profile.append(obj)

In [14]:
bulk_profile;

In [16]:
Profile.objects.all().delete()

(10,
 {'app.Profile_brand_likes': 0,
  'app.Profile_post_likes': 0,
  'app.Profile_brand_bookmarks': 0,
  'app.Profile_post_bookmarks': 0,
  'app.Profile_level_tested': 0,
  'app.Profile': 10})

In [17]:
Profile.objects.bulk_create(bulk_profile)

[<Profile: DazedKorea@getch.com>,
 <Profile: 8056LpjXNJdgwuj@getch.com>,
 <Profile: musinsacom@getch.com>,
 <Profile: rvd3065@getch.com>,
 <Profile: woodz_golmu@getch.com>,
 <Profile: fashionncom@getch.com>,
 <Profile: Lilith36910318@getch.com>,
 <Profile: SeoulStore_twt@getch.com>,
 <Profile: 4buynsell@getch.com>,
 <Profile: kanghui__@getch.com>,
 <Profile: agonyman@getch.com>,
 <Profile: i_ilyo@getch.com>,
 <Profile: jung_sang_won@getch.com>,
 <Profile: thin_krungy0314@getch.com>,
 <Profile: _feelthat2NE1@getch.com>,
 <Profile: Denny19871207@getch.com>,
 <Profile: yogipower@getch.com>,
 <Profile: DongWoonK@getch.com>,
 <Profile: pat5007@getch.com>,
 <Profile: ZENITH_NEWS@getch.com>,
 <Profile: tommygnosys@getch.com>,
 <Profile: FuxxingFabulous@getch.com>,
 <Profile: stussygo@getch.com>,
 <Profile: CorsoComoSeoul@getch.com>,
 <Profile: MARKO_CMO@getch.com>,
 <Profile: MrRaonKim@getch.com>,
 <Profile: makemesmile2030@getch.com>,
 <Profile: 6_closet@getch.com>,
 <Profile: allthekshop@ge

# Hashtag model

In [18]:
from app.models import Hashtag
#Hashtag.objects.all().delete()

In [19]:
hashtags = None
with open('hashtags.json', encoding='UTF-8-sig') as f:
    hashtags = json.load(f)

In [20]:
bulk_hashtags = []
for _tag in hashtags:
    obj = Hashtag()
    obj.hashtag = _tag
    bulk_hashtags.append(obj)

Hashtag.objects.bulk_create(bulk_hashtags)

[<Hashtag: 지창욱>,
 <Hashtag: JOOYOUNG>,
 <Hashtag: 迪丽热巴>,
 <Hashtag: 주정정>,
 <Hashtag: 와인스타그램>,
 <Hashtag: 와디즈>,
 <Hashtag: BGT2019>,
 <Hashtag: 제페토뷔>,
 <Hashtag: MLBB>,
 <Hashtag: speedpainting>,
 <Hashtag: 마영전_포코스>,
 <Hashtag: Choice>,
 <Hashtag: 하이컷스타>,
 <Hashtag: 호리즌>,
 <Hashtag: 늦었지만>,
 <Hashtag: Filson백팩>,
 <Hashtag: 클라터뮤젠>,
 <Hashtag: 신민재>,
 <Hashtag: 사무라이쇼다운m>,
 <Hashtag: 토도바쿠>,
 <Hashtag: 스트레칭>,
 <Hashtag: COLD>,
 <Hashtag: nyfw>,
 <Hashtag: 고야드지갑>,
 <Hashtag: الزلفي>,
 <Hashtag: 뱅앤올룹슨>,
 <Hashtag: 최진리>,
 <Hashtag: 강다니엘화보>,
 <Hashtag: 남자발바닥>,
 <Hashtag: 부탁>,
 <Hashtag: 교배>,
 <Hashtag: SNIDEL>,
 <Hashtag: パクシネ>,
 <Hashtag: Montblanc>,
 <Hashtag: 체전>,
 <Hashtag: 오랑씨>,
 <Hashtag: 5기머스터_부적>,
 <Hashtag: 주지훈>,
 <Hashtag: TOMFORD>,
 <Hashtag: 韓国ファッション>,
 <Hashtag: 버킷햇>,
 <Hashtag: 호시>,
 <Hashtag: 들켰어양>,
 <Hashtag: 아디다스정품>,
 <Hashtag: 물떼새격자무늬>,
 <Hashtag: 호비생일이벵>,
 <Hashtag: 헤르조그드뫼론>,
 <Hashtag: 두유워너비>,
 <Hashtag: 명품셔츠>,
 <Hashtag: kingofkpop>,
 <Hashtag: 리틀그라운드>,
 <Hashtag: LiveinLevis

In [29]:
cols = ['hashtag']
with open('dbinit_hashtags.csv', 'w', encoding='UTF-8') as f:
    writer = csv.DictWriter(f, fieldnames=cols)
    writer.writeheader()
    for data in dbinit_hashtags:
        writer.writerow(data['fields'])

# Feed model

In [25]:
from app.models import Brand, Profile, Hashtag, Feed

In [26]:
feeds = None
with open('feeds.json', encoding='UTF-8-sig') as f:
    feeds = json.load(f)

In [45]:
Feed.objects.all().delete()

(40, {'app.Feed_hashtags': 30, 'app.Feed': 10})

In [46]:
bulk_feeds = []

for i, _feed in enumerate(tqdm(feeds[:])):
    obj = Feed()
    obj.id = i+1
    obj.membership = Brand.objects.get(name=_feed['membership'])
    obj.author = Profile.objects.get(user__email=_feed['author']+'@getch.com')
    obj.timestamp = str(pd.Timestamp(_feed['created_at']).tz_localize(None))
    obj.nlikes = _feed['nlikes']
    obj.content = _feed['content']
    
    if 'feed_image' in _feed:
        obj.feed_image = 'feed_images/' + _feed['membership'] + '/' + _feed['author'] + '@getch.com' + '/' + _feed['feed_image'].split('/')[-1]
    
    bulk_feeds.append(obj)

KeyboardInterrupt: 

In [43]:
Feed.objects.bulk_create(bulk_feeds)

[<Feed: 2019-07-18 10:14:50.723664 DazedKorea@getch.com>,
 <Feed: 2019-07-18 10:14:50.723664 8056LpjXNJdgwuj@getch.com>,
 <Feed: 2019-07-18 10:14:50.723664 musinsacom@getch.com>,
 <Feed: 2019-07-18 10:14:50.723664 rvd3065@getch.com>,
 <Feed: 2019-07-18 10:14:50.723664 woodz_golmu@getch.com>,
 <Feed: 2019-07-18 10:14:50.723664 musinsacom@getch.com>,
 <Feed: 2019-07-18 10:14:50.723664 rvd3065@getch.com>,
 <Feed: 2019-07-18 10:14:50.723664 fashionncom@getch.com>,
 <Feed: 2019-07-18 10:14:50.723664 Lilith36910318@getch.com>,
 <Feed: 2019-07-18 10:14:50.723664 fashionncom@getch.com>]

In [44]:
for i, _feed in enumerate(tqdm(feeds[:10])):
    obj = Feed.objects.get(pk=i+1)
    obj.hashtags.set(Hashtag.objects.filter(hashtag__in=_feed['hashtags']))

In [81]:
dbinit_feeds = []

for pk, feed in enumerate(tqdm(feeds)):
    membership_pk = Brand.objects.get(name=feed['membership']).pk
    author_pk = Profile.objects.get(user__email=feed['author']+'@getch.com').pk
    hashtag_pks = list(Hashtag.objects.filter(hashtag__in=feed['hashtags']).values_list('pk', flat=True))
    
    fields = {
        'membership': membership_pk, 
        'author': author_pk, 
        'timestamp': str(pd.Timestamp(feed['created_at']).tz_localize(None)), 
        'nlikes': feed['nlikes'], 
        'content': feed['content'], 
        'hashtags': hashtag_pks
    }
    
    if 'feed_image' in feed:
        fields['feed_image'] = 'feed_images/' + feed['membership'] + '/' + feed['author'] + '/' + feed['feed_image'].split('/')[-1]
    
    dbinit_feeds.append({
        'model': 'app.feed', 
        'pk': pk+1, 
        'fields': fields
    })    

In [88]:
dbinit_feeds;

In [83]:
with open('dbinit_feeds.json', 'w') as f:
    json.dump(dbinit_feeds, f)

In [1]:
from app.models import Feed

In [27]:
res = Feed.objects.filter(hashtags__hashtag__in=['nike', 'adidas']); res

<QuerySet [<Feed: 2019-04-09 10:19:51 yogipower@getch.com>, <Feed: 2019-01-30 06:25:19 _MasterShoe_@getch.com>, <Feed: 2019-05-19 13:58:55 helloanimals@getch.com>, <Feed: 2019-03-19 12:24:30 sellittoyou@getch.com>, <Feed: 2019-04-12 13:51:45 ddungbom_mino@getch.com>, <Feed: 2019-01-02 19:01:14 AidenLee862@getch.com>, <Feed: 2019-02-04 03:42:22 altchang@getch.com>, <Feed: 2019-06-19 09:33:35 midasmarin@getch.com>, <Feed: 2019-05-05 08:40:28 chulhonp@getch.com>, <Feed: 2019-04-12 05:47:52 ddungbom_mino@getch.com>, <Feed: 2019-04-23 09:11:25 URmystar_0330@getch.com>, <Feed: 2019-03-03 19:11:40 Kim_JiSooYG1@getch.com>, <Feed: 2019-04-27 00:15:33 midasmarin@getch.com>, <Feed: 2019-03-25 02:03:43 getitk@getch.com>, <Feed: 2019-04-23 07:27:27 URmystar_0330@getch.com>, <Feed: 2019-03-20 03:29:47 pyobongbong@getch.com>, <Feed: 2019-05-04 07:06:57 midasmarin@getch.com>, <Feed: 2019-04-27 02:26:06 Eternal_1229@getch.com>, <Feed: 2019-04-17 14:09:46 hyun8034@getch.com>, <Feed: 2019-04-12 14:27:28 

In [28]:
res[100].content

'미노야 널 좋아해💗\n\n#WINNER \n#위너 \n#송민호\n#MINO\n#아디다스\n#adidas\n@official_mino_ https://t.co/7hq2HWcUgU'

In [29]:
res[100].hashtags.all()

<QuerySet [<Hashtag: MINO>, <Hashtag: 송민호>, <Hashtag: 위너>, <Hashtag: adidas>, <Hashtag: 아디다스>, <Hashtag: WINNER>]>

In [1]:
import os

In [4]:
os.path.join('C://', 'test', 'kkk.jpg')

'C://test\\kkk.jpg'